In [1]:
!pip install jsonlines

In [1]:
import os
import re
import jsonlines
import random
import json
import pandas as pd
from typing import List, Tuple

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
input_file = "/content/drive/MyDrive/Colab Notebooks/dataset/trn.json"
cleaned_output_file = "/content/drive/MyDrive/Colab Notebooks/dataset/cleaned_dataset.json"
transformed_output_file = "/content/drive/MyDrive/Colab Notebooks/dataset/transformed_dataset.json"
fraction = 0.6
seed = 42

In [3]:
def load_dataset(input_file: str) -> List[dict]:
    """
    Carrega o dataset a partir de um arquivo JSONLines.

    Args:
        input_file (str): Caminho do arquivo JSONLines de entrada.

    Returns:
        List[dict]: Lista de dicionários com os dados do JSONLines.
    """
    try:
        with jsonlines.open(input_file, mode='r') as reader:
            return [line for line in reader]
    except Exception as e:
        raise ValueError(f"Erro ao carregar o arquivo: {e}")

In [4]:
def validate_inputs(input_file: str, fraction: float) -> None:
    """
    Valida os inputs para a função create_sample_dataset.
    """
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"O arquivo {input_file} não foi encontrado.")
    if not (0 < fraction <= 1):
        raise ValueError("A fração deve estar no intervalo (0, 1].")

In [5]:
def filter_empty_entries(data: pd.DataFrame) -> pd.DataFrame:
    """
    Remove entradas vazias do DataFrame.

    Args:
        data (pd.DataFrame): DataFrame contendo as colunas 'title' e 'content'.

    Returns:
        pd.DataFrame: DataFrame filtrado sem entradas vazias.
    """
    print(f"Linhas antes da filtragem: {len(data)}")
    filtered_data = data.dropna(subset=['title', 'content']).query("title != '' and content != ''")
    print(f"Linhas após a filtragem: {len(filtered_data)}")
    return filtered_data

In [6]:
html_entities = {
    "&quot;": '"',    # Aspas duplas
    "&apos;": "'",    # Aspas simples/apóstrofo
    "&amp;": "&",     # E comercial
    "&lt;": "<",      # Menor que
    "&gt;": ">",      # Maior que
    "&nbsp;": " ",    # Espaço não separável
    "&ldquo;": "“",   # Aspas duplas esquerda
    "&rdquo;": "”",   # Aspas duplas direita
    "&lsquo;": "‘",   # Aspas simples esquerda
    "&rsquo;": "’",   # Aspas simples direita
    "&copy;": "©",    # Símbolo de copyright
    "&ndash;": "–",   # Meia-risca
    "&mdash;": "—",   # Travessão
    "&x201C": "“",    # Aspas duplas esquerda
    "&x201D": "”",    # Aspas duplas direita
    "&x2019": "’",    # Apóstrofo tipográfico
    "&hellip;": "…",  # Reticências
}
def remove_html_entities(text, entity_map):
    """
    Limpa o texto, removendo caracteres especiais.

    Args:
        text (str): Texto que a ser limpo;
        entity_map: Mapa de entidades a serem removidas do texto
    Returns:
        O texto limpo em formato string.
    """

    for entity in entity_map.keys():
        text = text.replace(entity, "")

    text = re.sub(r"&x[0-9A-Fa-f]+;?", "", text)
    return text

In [7]:
def create_sample_dataset(input_file: str, output_file: str, fraction: float, seed: int) -> Tuple[int, int]:
    """
    Extrai uma fração aleatória do dataset, mantém 'title' e 'content',
    remove entradas vazias e salva em um novo arquivo.

    Args:
        input_file (str): Caminho do arquivo JSONLines de entrada.
        output_file (str): Caminho do arquivo JSONLines de saída.
        fraction (float): Fração do dataset a ser extraída.
        seed (int): Semente para garantir reprodutibilidade.

    Returns:
        Tuple[int, int]: Total de linhas no dataset final e total de registros originais.
    """
    try:
        validate_inputs(input_file, fraction)

        print("Carregando o dataset...")
        data = load_dataset(input_file)

        if not data:
            print("O arquivo de entrada está vazio.")
            return 0, 0

        random.seed(seed)

        total_samples = len(data)
        num_samples = max(1, int(total_samples * fraction))
        print(f"Total de registros no arquivo original: {total_samples}")
        print(f"Número de registros selecionados: {num_samples}")
        sampled_data = random.sample(data, num_samples)

        print("Processando os dados...")
        mapa = str.maketrans("", "", "@#*%$;[]'/")
        processed_data = [
            {"title": remove_html_entities(entry.get("title", "").translate(mapa).replace("&quot", '').replace('\"', '') , html_entities), "content": remove_html_entities(entry.get("content", "").translate(mapa).replace("&quot", '').replace('\"', '') , html_entities) }
            for entry in sampled_data
        ]

        df = pd.DataFrame(processed_data)
        df = filter_empty_entries(df)

        print(f"Salvando o dataset em {output_file}...")
        df.to_json(output_file, orient="records", lines=True, force_ascii=False)
        print(f"Dataset tratado salvo com sucesso em: {output_file}")

        total_final = len(df)
        return total_final, total_samples, df

    except Exception as e:
        print(f"Erro durante o processamento: {e}", exc_info=True)
        return 0, 0

In [8]:
def analyze_dataset(file_path: str) -> None:
    """
    Analisa um arquivo JSONLines para verificar se todas as linhas seguem o formato esperado.

    Args:
        file_path (str): Caminho para o arquivo JSONLines.
    """
    total_lines = 0
    valid_lines = 0
    missing_title = 0
    missing_content = 0
    empty_title = 0
    empty_content = 0

    try:
        print(f"Analisando o arquivo: {file_path}")
        with jsonlines.open(file_path, mode='r') as reader:
            for line in reader:
                total_lines += 1

                has_title = "title" in line
                has_content = "content" in line

                if has_title and has_content:
                    valid_lines += 1

                    if not line["title"].strip():
                        empty_title += 1
                    if not line["content"].strip():
                        empty_content += 1
                else:
                    if not has_title:
                        missing_title += 1
                    if not has_content:
                        missing_content += 1

        # Exibir resumo
        print("Resumo da análise do dataset:")
        print(f"Total de linhas: {total_lines}")
        print(f"Linhas válidas: {valid_lines}")
        print(f"Linhas com 'title' ausente: {missing_title}")
        print(f"Linhas com 'content' ausente: {missing_content}")
        print(f"Linhas com 'title' vazio: {empty_title}")
        print(f"Linhas com 'content' vazio: {empty_content}")
        print(f"Linhas inválidas: {total_lines - valid_lines}")

    except Exception as e:
        print(f"Erro ao analisar o dataset: {e}", exc_info=True)

In [9]:
def transform_dataset(df, output_file):
    """
    Formata o dataset para o formato que o modelo esta esperando para ser treinado.

    Args:
        df: Dataset ja processado e limpo.
        output_file: Caminho onde o arquivo transformado será salvo
    """
    transformed_data = {
        "instruction": ["SUMMARIZE THIS PRODUCTS"] * len(df),
        "input": df["title"].tolist(),
        "output": df["content"].tolist()
    }

    # Salvar o novo dataset em um arquivo JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(transformed_data, f, ensure_ascii=False, indent=4)

In [10]:
analyze_dataset(input_file)

print("Iniciando o processamento do dataset...")
total_final, total_original, df = create_sample_dataset(input_file, cleaned_output_file, fraction, seed)

if total_original > 0:
    final_percentage = (total_final / total_original) * 100
    print(
        f"Processamento concluído. Total de registros no dataset final: {total_final} "
        f"({final_percentage:.2f}% do total original)."
    )
else:
    print("O dataset original estava vazio ou houve um erro no processamento.")

analyze_dataset(cleaned_output_file)

transform_dataset(df, transformed_output_file)

Analisando o arquivo: C:/Projects/FIAP_IA_Fro_Devs/TechChallenge-Fase3/Tests/LF-Amazon/trn-folder/trn.json
Resumo da análise do dataset:
Total de linhas: 2248619
Linhas válidas: 2248619
Linhas com 'title' ausente: 0
Linhas com 'content' ausente: 0
Linhas com 'title' vazio: 126834
Linhas com 'content' vazio: 749901
Linhas inválidas: 0
Iniciando o processamento do dataset...
Carregando o dataset...
Total de registros no arquivo original: 2248619
Número de registros selecionados: 1349171
Processando os dados...
Linhas antes da filtragem: 1349171
Linhas após a filtragem: 834306
Salvando o dataset em C:/Projects/FIAP_IA_Fro_Devs/TechChallenge-Fase3/Tests/LF-Amazon/trn-folder/cleaned_dataset.json...
Dataset tratado salvo com sucesso em: C:/Projects/FIAP_IA_Fro_Devs/TechChallenge-Fase3/Tests/LF-Amazon/trn-folder/cleaned_dataset.json
Processamento concluído. Total de registros no dataset final: 834306 (37.10% do total original).
Analisando o arquivo: C:/Projects/FIAP_IA_Fro_Devs/TechChallenge-